In [7]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Read input data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [8]:
## Hyper parameters
EPOCHS = 1000
LEARNING_RATE = 0.1
N_BATCHES = 100


In [9]:
## Construct model
class LearningModel:
    def __init__(self, sess):
        self.sess = sess
        self.build_model()
        
    def build_model(self):
        self.X = tf.placeholder(tf.float32, [None, 784])
        self.Y = tf.placeholder(tf.float32, [None, 10])

        W = tf.get_variable('W', shape=[784, 10], initializer=tf.contrib.layers.xavier_initializer())
        b = tf.Variable(tf.zeros([10]))

        self.model = tf.matmul(self.X, W) + b

        # define cost and optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.model, labels=self.Y))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE).minimize(self.cost)

        # define accuracy
        prediction = tf.equal(tf.argmax(self.model, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
        
    def get_accuracy(self, x_test, y_test):
        return self.sess.run(self.accuracy, feed_dict={self.X:x_test, self.Y:y_test})
    
    def do_train(self, x_test, y_test):
        return self.sess.run([self.optimizer, self.cost], feed_dict={self.X:x_test, self.Y:y_test})

In [10]:
## Training
with tf.Session() as sess:
    mymodel = LearningModel(sess)
    sess.run(tf.global_variables_initializer())

    total_cost = 0
    t_start = time.time()
    for i in range(EPOCHS):
        opt, cost_val = mymodel.do_train(mnist.train.images, mnist.train.labels)
        total_cost += cost_val
    
    print("Traing time: %f sec" % (time.time() - t_start))
    print('Average Cost: ', '{}'.format((total_cost/EPOCHS).astype(np.float32)))

    ## Test
    accuracy = mymodel.get_accuracy(mnist.test.images, mnist.test.labels)
    print('accuracy: ', accuracy)

Traing time: 56.756581 sec
Average Cost:  0.46801507472991943
accuracy:  0.9095
